# Testing `nakeds.py`

In [ ]:
MARKET = 'NSE'

In [2]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [3]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [4]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [5]:
# Get the host, port, cid
from engine import Vars

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [6]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_margins.pkl', 'df_opt_prices.pkl', 'df_opts.pkl',
       'df_symlots.pkl', 'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl',
       'qopts.pkl'], dtype='<U18')

## Sample individual `nakeds`

In [7]:
from nakeds import get_nakeds

### Get sort by highest volatility

In [8]:
cols = ['symbol', 'undPrice', 'iv', 'margin', 'putOpenInterest', 'callOpenInterest']
df_unds.sort_values('iv', ascending=False)[cols]

,symbol,undPrice,iv,margin,putOpenInterest,callOpenInterest
11,IDEA,9.95,0.92,"680,801.83",1154,2204
137,IBULHSGFI,193.30,0.81,"601,300.81",1277,2916
16,RBLBANK,231.80,0.62,"616,980.01",1894,2107
31,INDUSINDB,896.25,0.59,"733,600.00",5746,6113
36,INFRATEL,239.55,0.57,"662,446.39",1269,1942
...,...,...,...,...,...,...
129,MARICO,389.65,0.27,"788,935.97",357,1255
55,TCS,"2,709.45",0.27,"817,184.99",6972,10967
24,DABUR,501.50,0.26,"627,788.73",1938,3045
133,HINDUNILV,"2,127.75",0.25,"656,640.01",4250,6888


### Pick up a symbol

In [30]:
SYMBOL = 'NIFTY50'
drop_cols = ['conId', 'contract', 'secType', 'comm', 'close', 'last', 'und_iv']

In [31]:
df = get_nakeds(MARKET=MARKET,
                SYMBOL=SYMBOL,
                EARLIEST=True,
                RECALC_UNDS=True,
                SAVE=False
                )


nakeds started at 07-Dec-2020 17:53:35


und:        0%|                         | 0/1 [00:00<?, ?it/s]                  


nse underlyings started at 07-Dec-2020 17:53:35



Error: wif could not complete for contract: NIFTY50
Try by increasing FILL_DELAY from > 5.5 secs


...nse underlyings took: 00:00:10 seconds


dfrqs started at 07-Dec-2020 17:53:46

...dfrqs took: 00:00:00 seconds



price:      0%|                         | 0/186 [00:00<?, ?it/s]                


NSE nakeds option price started at 07-Dec-2020 17:53:47


margin:     0%|                         | 0/186 [00:00<?, ?it/s]                


...NSE nakeds option price took: 00:00:28 seconds


NSE naked option margins started at 07-Dec-2020 17:54:16



...NSE naked option margins took: 00:00:06 seconds


...nakeds took: 00:00:47 seconds



In [32]:
df1 = df.drop(drop_cols, 1)

m1 = df1.ask>0
m2 = df1.sdMult>1.2
df1[m1&m2].head()

,symbol,expiry,dte,right,strike,sd_lmt,fallrise,fr_sd,undPrice,lot,iv,qty,margin,bid,ask,sdMult,prop,intrinsic,timevalue,price,expPrice,rom,expRom
9,NIFTY50,20201210,3,P,"13,050.00","13,098.43","12,030.65",6.47,"13,354.20",75.00,0.20,75.00,"183,590.56",15.15,15.20,1.24,0.79,0.00,15.15,15.15,15.15,0.74,0.74
10,NIFTY50,20201210,3,P,"13,000.00","13,098.43","12,030.65",6.47,"13,354.20",75.00,0.21,75.00,"180,221.12",11.85,11.95,1.38,0.83,0.00,11.90,11.90,14.15,0.59,0.70
12,NIFTY50,20201210,3,P,"12,950.00","13,098.43","12,030.65",6.47,"13,354.20",75.00,0.22,75.00,"176,753.10",9.20,9.30,1.51,0.87,0.00,9.25,9.25,13.95,0.46,0.70
13,NIFTY50,20201210,3,P,"12,900.00","13,098.43","12,030.65",6.47,"13,354.20",75.00,0.23,75.00,"173,179.77",7.60,7.65,1.62,0.89,0.00,7.60,7.60,13.75,0.39,0.70
14,NIFTY50,20201210,3,P,"12,850.00","13,098.43","12,030.65",6.47,"13,354.20",75.00,0.24,75.00,"169,560.34",6.20,6.30,1.72,0.91,0.00,6.25,6.25,13.60,0.32,0.70
